In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('titanic').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
df = spark.read.csv('titanic.csv',inferSchema=True,header=True)

In [6]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [7]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [8]:
my_cols = df.select([
 'Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [9]:
final_data = my_cols.na.drop()

In [10]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder, StringIndexer)

In [11]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [12]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [13]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],
                            outputCol='features')

In [14]:
from pyspark.ml import Pipeline

In [15]:
lr_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [16]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                            gender_encoder,embark_encoder,
                            assembler,lr_titanic])

In [17]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [18]:
fit_model = pipeline.fit(train_data)

In [19]:
results = fit_model.transform(test_data)

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
Bi_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [23]:
AUC = Bi_eval.evaluate(results)

In [24]:
AUC

0.7387820512820513